# Assignment 2
In this assignment you will be writing netunicorn experiments to measure 'latency-under-load'.  

Please submit a PDF verion of this notebook with all the cells evaluated and answers displayed.  

In [ ]:
import os
import time
import random
import time
import requests 
import re
import logging
import subprocess
from subprocess import Popen
from sys import platform
import os, sys
import logging
import json
import threading
import numpy as np

import subprocess
import time
import signal
from typing import Dict

from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher


from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest
from netunicorn.library.tasks.measurements.ping import Ping
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node
from netunicorn.base.task import Failure, Task, TaskDispatcher
from netunicorn.base import Result, Failure, Success, Task, TaskDispatcher
from netunicorn.base.architecture import Architecture
from netunicorn.base.nodes import Node

from typing import Dict
from typing import Optional
from enum import IntEnum
from datetime import datetime

from returns.pipeline import is_successful
from returns.result import Failure

### Helper Functions
We have defined two helper functions for executing pipelines and displaying the results from the pipeline execution that you may use.

In [ ]:
def execute_pipeline(pipeline, working_node, experiment_label):
    client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
    nodes = client.get_nodes()
    working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)
    experiment = Experiment().map(pipeline, working_nodes)

    try:
        client.delete_experiment(experiment_label)
    except RemoteClientException:
        pass

    # Prepare Experiment
    client.prepare_experiment(experiment, experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status == ExperimentStatus.READY:
            break
        time.sleep(20)

    time.sleep(5)

    # Execute Experiment
    client.start_execution(experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status != ExperimentStatus.RUNNING:
            break
        time.sleep(20)
    return info

In [ ]:
def display_results(info):
    # Get Results
    for report in info.execution_result:
        print(f"Node name: {report.node.name}")
        print(f"Error: {report.error}")

        if report.result is None:
            print("report.result is EMPTY..")
            continue

        result, log = report.result  # report stores results of execution and corresponding log

        # result is a returns.result.Result object, could be Success of Failure
        print(f"Result is: {type(result)}")
        if is_successful(result):
            data = result.unwrap()
        else:
            data = result.failure()
        try:
            for key, value in data.items():
                print(f"{key}: {value}")
        except:
            print(f"No attribute 'items' in result")

        # we also can explore logs
        for line in log:
            print(line.strip())
        print()

### Task 0
Set your credentials for accessing the UCSB netUnicorn deployment below. View the working nodes and choose 1 working node that you will be using for this assignment.

In [ ]:
# TODO: fill in your login / password below
netunicorn_login = '<fill me in>'
netunicorn_password = '<fill me in>'

NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', netunicorn_login)
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', netunicorn_password)

client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)

In [ ]:
# TODO: fill in the name of your working node
working_node = '<fill me in>'

### Task 1
Create a pipeline to collect latency data to Google's public DNS (8.8.8.8) from your working node using the [Ping](https://github.com/netunicorn/netunicorn-library/blob/main/tasks/measurements/ping.py) task.   
Report the average latency over 5 ICMP probes to the address 8.8.8.8

In [ ]:
# TODO: fill in the correct arguments for the Ping() task
pipeline = (
    Pipeline()
    .then(Ping())
)

In [ ]:
info = execute_pipeline(pipeline, working_node, 'ping_experiment')
display_results(info)

In [ ]:
# TODO: Fill the latency samples array with the latency values returned by the Ping task
latency_samples = [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
]
print("Average Latency: {:.4f} ms".format(np.mean(latency_samples)))

### Task 2
Now we will define a new task, StartBackgroundPing that behaves similar to the Ping task, however it starts the process in the background, so further tasks in the pipeline will execute in parallel with our latency measurements.  

Implement the **run()** function in the **StartBackgroundPingLinuxImplementation** class

In [ ]:
class StartBackgroundPing(TaskDispatcher):
    def __init__(self, address: str, filepath: str, timeout: int = 120, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.address = address
        self.filepath = filepath
        self.timeout = timeout
        self.linux_implementation = StartBackgroundPingLinuxImplementation(
            address=self.address,
            filepath=self.filepath,
            timeout=self.timeout,
            *args,
            **kwargs
        )

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation
        raise NotImplementedError(
            f'StartBackgroundPing is not implemented for architecture: {node.architecture}'
        )

class StartBackgroundPingLinuxImplementation(Task):
    def __init__(self, address: str, filepath: str, timeout: int = 120, *args, **kwargs):
        self.address = address.strip()
        self.filepath = filepath
        self.timeout = timeout
        super().__init__(*args, **kwargs)
    
    # TODO: implement the run() function
    def run(self):
        # 1. open self.filepath in write mode
        outfile = "<fill me in>"
        # 2. set the `args` argument for Popen to execute the command `ping -t self.timeout`. 
        # 3. set the optional arguments: stdout, stderr to the file object we opened for self.filepath
        proc = subprocess.Popen("<fill me in>")

        time.sleep(2)
        if (exit_code := proc.poll()) is None:  # not finished yet
            return Success(proc.pid)
        return Failure(f"StartBackgroundPing failed with return code {exit_code}")

Below, we define a complementary task, StopBackgroundPing to kill our ping process so it doesn't always execute until the timeout. By specifying an optional argument `name` to our StartBackgroundPing task, we can look up the results for this task using the `self.previous_steps` object, to retrieve the PID of the ping process that we return in the StartBackgroundPing task.

In [ ]:
class StopBackgroundPing(TaskDispatcher):
    def __init__(self, start_ping_task_name : str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.start_ping_task_name = start_ping_task_name
        self.linux_implementation = StopBackgroundPingLinuxImplementation(
            ping_task_name=self.start_ping_task_name,
            *args,
            **kwargs,
        )
    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation

        raise NotImplementedError(
            f'StopBackgroundPing is not implemented for {node.architecture}'
        )

class StopBackgroundPingLinuxImplementation(Task):
    requirements = ["sudo apt-get update", "sudo apt-get install -y tcpdump", "sudo apt-get install -y procps"]

    def __init__(self, ping_task_name: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.ping_task_name = ping_task_name

    def run(self):
        signal.signal(signal.SIGCHLD, signal.SIG_IGN)
        pid = self.previous_steps.get(self.ping_task_name, [Failure("Named StartBackgroundPing not found")])[-1]
        if isinstance(pid, Failure):
            return pid

        pid = pid.unwrap()
        return subprocess.check_output(["kill", str(pid)])

### Task 3
Now we will define a new task, GetFileContents that will output the results that we wrote to a file in the StartBackgroundPing task. This task simply prints the content of a file to stdout.

Implement the **run()** function in the **GetFileContents** class

In [ ]:
class GetFileContents(TaskDispatcher):
    def __init__(self, filepath: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.linux_implementation = GetFileContentsLinuxImplementation(
            filepath=filepath, name=self.name
        )

    def dispatch(self, node: Node) -> Task:
        if node.architecture in {Architecture.LINUX_AMD64, Architecture.LINUX_ARM64}:
            return self.linux_implementation
        raise NotImplementedError(
            f"GetFileContents is not implemented for architecture: {node.architecture}"
        )

class GetFileContentsLinuxImplementation(Task):
    def __init__(self, filepath: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filepath = filepath

    # TODO: implement the run() function
    def run(self):
        # 1. set the arguments for subprocess.run to exectue the command `cat self.filepath`
        # 2. set the optional argument `capture_output` to True so that stdout will be captured
        result = subprocess.run("<fill me in>")
        
        if result.returncode != 0:
            return Failure(
                result.stdout.decode("utf-8").strip()
                + "\n"
                + result.stderr.decode("utf-8").strip()
            )

        return result.stdout.decode("utf-8")

### Task 4
Collect latency samples to 8.8.8.8 using the new tasks we've defined. The pipeline has already been defined for you below, but requires that the StartBackgroundPing and GetFileContents tasks are implemented correctly. Plot the results. The x-axis should be the ICMP sequence number (icmp_seq), and the y-axis should the latency in milliseconds. You should observe similar values to what you observed from Task 1.

In [ ]:
pipeline = (
    Pipeline()
    .then(StartBackgroundPing(address='8.8.8.8', filepath="/tmp/background_ping_results1.txt", name="background_ping"))
    .then(SleepTask(30))
    .then(StopBackgroundPing(start_ping_task_name="background_ping"))
    .then(SleepTask(2))
    .then(GetFileContents(filepath="/tmp/background_ping_results1.txt"))
)

In [ ]:
info = execute_pipeline(pipeline, working_node, 'background_ping_experiment')
display_results(info)

In [ ]:
# TODO
# 1. Parse netUnicorn output
# 2. Graph Results

### Task 5
Now we will use the SpeedTest task to induce extra load on the network, and simultaneously collect our lateny samples. The pipeline has already been defined for you below, but requires that the StartBackgroundPing and GetFileContents tasks are implemented correctly. Plot the results alongside the results from Task 4. The x-axis should be the ICMP sequence number (icmp_seq), and the y-axis should the latency in milliseconds.

In [ ]:
pipeline = (
    Pipeline()
    .then(StartBackgroundPing(address='8.8.8.8', filepath="/tmp/background_ping_results2.txt", name="background_ping_under_load"))
    .then(SpeedTest())
    .then(StopBackgroundPing(start_ping_task_name="background_ping_under_load"))
    .then(SleepTask(2))
    .then(GetFileContents(filepath="/tmp/background_ping_results2.txt"))
)

In [ ]:
info = execute_pipeline(pipeline, working_node, 'background_ping_under_load_experiment')
display_results(info)

In [ ]:
# TODO
# 1. Parse netUnicorn output
# 2. Graph Results alongside Task 4

### Task 6
Do you observe a difference in latencies between Task 4 and Task 5? If so, what is the reason that we observe an increased latency when we are simultaneously running a speedtest?

Explain your answer to Task 6 below.